> 本页中我们介绍在python中写单元测试的数种方法，并比较优劣
> 1. 直接在main函数中写测试；
> 2. 使用标准库`unittest`；
> 3. 使用第三方库`pytest`。

## 1. 需要测试的函数
-------------

这里先定义我们需要测试的函数，以Fibonacci函数为例:

In [1]:
def fibonacci(n):
    if n == 1 or n == 2:
        return 1
    elif n >= 3:
        return fibonacci(n-1) + fibonacci(n-2)
    else:
        raise ValueError("n can't be negative")

在这个函数中，我们使用了fibonacci的递归定义：

$$
\left \{
\begin{align}
\mathrm{Fib}_{n} &= \mathrm{Fib}_{n - 1} + \mathrm{Fib}_{n - 2} , \; n \ge 3 \\
\mathrm{Fib}_2 &= \mathrm{Fib}_1 = 1 \\
\end{align}
\right .
$$

在上面的函数中，我们实际上做了三件事：

1. 若 $n=1$ 或者 $n=0$ 的话，直接返回1；
2. 若 $n \ge 2$ ，返回递归定义的结果；
3. 其余情况 $n \le 0$ ，直接抛出异常。

使用上面定义的函数，我们打印前10个Fibonacci数：

In [2]:
print("Fibonacci series until 10")
for n in range(1, 11):
    print(f"n = {n:<2d}: {fibonacci(n):4d}")

Fibonacci series until 10
n = 1 :    1
n = 2 :    1
n = 3 :    2
n = 4 :    3
n = 5 :    5
n = 6 :    8
n = 7 :   13
n = 8 :   21
n = 9 :   34
n = 10:   55


由上面函数中所作的三件事，我们可以很容易的想到，如果需要测试，就也要针对这三个分支分别测试：

1. 测试在 $n=1$ 或者 $n=0$ 的情况下，返回结果为1；
2. 测试在 $n \ge 2$ 的情况下，返回结果确实为递归定义的要求；
3. 测试在 $n \le 0$ 的时候，函数会正确的抛出异常。

## 2. 在python文件的main模块中测试
-------------------

每个python文件都可以在文件尾插入main函数，从而在该文件以main模块形式导入的时候，选择性运行。所以我们可以利用python语言的这一特性，将测试代码写入main函数之中，这样我们可以通过直接运行该文件的形式，运行测试。

具体的形式，例如如下：

```python

some functional codes here 
...

if __name__ == '__main__':
    some test codes here
    ...

```

在我们这个例子里面，我们

* 配合python内置的`assert`语句来测试函数返回值；
* 利用`try...except...`代码块来测试抛出的异常。

最后使用下面的代码来完成测试：

In [3]:
if __name__ == '__main__':
    assert fibonacci(1) == 1, "fibonacci(1) is not equale to 1"
    assert fibonacci(2) == 1, "fibonacci(2) is not equale to 1"
    assert fibonacci(3) == fibonacci(1) + fibonacci(2), "fibonacci(3) is not equal to fibonacci(1) + fibonacci(2)"
    
    try:
        fibonacci(-1)
    except ValueError:
        pass
    else:
        raise Exception("fibonacci(-1) does not correctly raise ValueError")
    print("Every thing is Ok")

Every thing is Ok


完整的代码请见同目录下文件[`test_main.py`](test_main.py)，可以使用下面的代码运行样例。

In [4]:
!python test_main.py

Every thing is Ok


使用main函数的形式来完成测试的优点是简单易学，但是也有几个缺点：

* 测试代码的运行需要将每个文件单独以main模块的方式运行，如果需要测试的文件很多，则需要多次运行python解释器；
* 除非出错，否则没有输出信息，不方便追踪；
* 使用`try...except...`的方式，来测试抛出异常的方式，不够直观明了。

## 3. 使用标准库`unittest`进行测试
------------------

python标准库中已经内置了一套[`unittest`](https://docs.python.org/3.6/library/unittest.html)，用来帮助python用户来完成单元测试的任务。使用`unitest`库，有下面的好处：

* 内置的多种断言方式，可以用来检测布尔型、整型、浮点型等；
* 输出标准的log信息，帮便追踪；
* 更结构化的方式组织测试代码；
* 测试异常情形的方式更加简洁。

`unittest`的使用方式，基本上是以继承基类的方式进行：

```python
import unittest

class MyTest(unittest.TestCase):
    
    def test_something(self):
        some codes here
        ...
```

其中`MyTest`是测试集合类，一般总是继承`unittest.TestCase`。之后，`MyTest`中所有以`test_`打头的方法都是单个测试用例。我们这里展示完整的代码：

In [5]:
import unittest

class FibonacciTest(unittest.TestCase):
    
    def test_fibonacci_less_than_three(self):
        self.assertEqual(fibonacci(1), 1)
        self.assertEqual(fibonacci(2), 1)
        
    def test_fibonacci_recursive(self):
        for n in range(3, 11):
            self.assertEqual(fibonacci(n), fibonacci(n-1) + fibonacci(n-2))
            
    def test_fibonacci_with_negative(self):
        with self.assertRaises(ValueError):
            _ = fibonacci(-1)
            

if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(FibonacciTest)
    unittest.TextTestRunner(verbosity=2).run(suite)

test_fibonacci_less_than_three (__main__.FibonacciTest) ... ok
test_fibonacci_recursive (__main__.FibonacciTest) ... ok
test_fibonacci_with_negative (__main__.FibonacciTest) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.004s

OK


完整例子也可以在[`test_unittest`](test_unittest.py)文件中找到，也可以在命令行里面运行，这里我们略去。

## 4. 使用第三方库`pytest`完成单元测试
-----------------

虽然`unittest`解决了直接把测试内容写在main函数里面的问题，但是它仍然存在自己的问题：

* 单元测试很冗长，不符合python追求简单的特点；
* 需要使用类继承的方法，测试不够直接；
* 测试用assert名称很多，有选择困难。

这些特点是由于`unittest`照搬了`xUnit`的体系，而`xUnit`最早的实现都是基于纯面向对象的语言，例如：Smalltalk，Java等。python的`unittest`的实现不免受上面这些影响，带有了太多面向对象的特征。

[`pytest`](https://docs.pytest.org)是一个更具有python本身特点的测试库，可以用更简洁的方式去完成测试。例如上面，第3部分相同功能的代码，在`pytest`中可以如下实现：

```python
import pytest

def test_fibonacci_less_than_three():
    assert fibonacci(1) == 1
    
def test_fibonacci_recursive():
    for n in range(3, 11):
        assert fibonacci(n) == fibonacci(n-1) + fibonacci(n-2)

def test_fibonacci_with_negative():
    with pytest.raises(ValueError):
            _ = fibonacci(-1)

```

我们发现代码相比于标准库`unittest`的实现要干净很多。全部代码可见：[test_pytest.py](test_pytest.py)，我们可以使用`pytest`的命令行工具运行：

In [6]:
!pytest -v test_pytest.py

============================= test session starts =============================
platform win32 -- Python 3.6.6, pytest-3.8.0, py-1.6.0, pluggy-0.7.1 -- D:\ProgramData\anaconda3\python.exe
cachedir: .pytest_cache
rootdir: D:\dev\github\my_python_experience\notebook, inifile:
plugins: remotedata-0.3.0, openfiles-0.3.0, doctestplus-0.1.3, arraydiff-0.2
collecting ... collected 3 items

test_pytest.py::test_fibonacci_less_than_three PASSED                    [ 33%]
test_pytest.py::test_fibonacci_recursive PASSED                          [ 66%]
test_pytest.py::test_fibonacci_with_negative PASSED                      [100%]

========================== 3 passed in 0.02 seconds ===========================


我们可以看到`pytest`几乎拥有`unittest`所有的优点，而没有`unittest`冗长的缺点。